In [ ]:
"""
逻辑回归：
    概念：解决一个二分类问题、逻辑回归的输入是线性回归的输出
    原理：
        1.输入：线性回归的输出
        2.激活函数：sigmod函数，把整体映射到【0，1】，在设置一个阈值进行分类判断
        3.损失：对数似然损失（借助log思想完成，根据真实值0、1两种情况进行划分）  线性回归中是最小二乘法。
        4.优化：提升原本属于1的概率、降低原本属于0的概率。

逻辑回归api介绍：
sklearn.linear_model.LogisticRegression(solver='liblinear', penalty=‘l2’, C = 1.0)
    solver可选参数:{'liblinear', 'sag', 'saga','newton-cg', 'lbfgs'}，表示可选的梯度下降法。
    默认: 'liblinear'；用于优化问题的算法。
    对于小数据集来说，“liblinear”是个不错的选择，而“sag”和'saga'对于大型数据集会更快。
    对于多类问题，只有'newton-cg'， 'sag'， 'saga'和'lbfgs'可以处理多项损失;“liblinear”仅限于“one-versus-rest”分类。
    penalty：正则化的种类
    C：正则化力度

默认将类别数量少的当做正例
LogisticRegression方法相当于 SGDClassifier(loss="log", penalty=" "),SGDClassifier实现了一个普通的随机梯度下降学习。而使用LogisticRegression(实现了SAG)

"""

In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

import joblib
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [50]:
# 1.获取数据
names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']

data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data", names=names)
data.head(10)
# data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [49]:
# 2.基本数据处理
# 2.1 缺失值处理
data = data.replace(to_replace="?", value=np.NaN)
data = data.dropna()
# 2.2 确定特征值,目标值
x = data.iloc[:, 1:10]
x.head()
y = data["Class"]
y.head()
# 2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [11]:
# 3.特征工程(标准化)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [40]:
# 4.机器学习(逻辑回归)
estimator = LogisticRegression()
estimator.fit(x_train, y_train)

# 4.1 模型保存
joblib.dump(estimator, "./model.pkl")

['./model.pkl']

In [41]:
# 4.3 模型加载
estimator = joblib.load(r"./model.pkl")

In [46]:
# 5.模型评估
# 5.1基本评估
y_predict = estimator.predict(x_test)
print(y_test, y_predict)
print(estimator.score(x_test, y_test))

389    2
32     4
272    4
655    2
271    2
      ..
296    2
585    2
256    2
43     4
691    4
Name: Class, Length: 171, dtype: int64 [2 4 4 2 2 2 2 2 2 2 2 2 2 4 2 2 4 4 4 2 4 2 4 4 4 2 4 2 2 2 2 2 4 2 2 2 4
 2 2 2 2 4 2 4 4 4 4 2 4 4 2 2 2 2 2 4 2 2 2 2 4 4 4 4 2 4 2 2 4 2 2 2 2 4
 2 2 2 2 2 2 4 4 4 2 4 4 4 4 2 2 2 4 2 4 2 2 2 2 2 2 4 2 2 4 2 2 4 2 4 4 2
 2 2 2 4 2 2 2 2 2 2 4 2 4 2 2 2 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 2 4
 2 2 4 4 4 2 2 4 4 2 4 4 2 2 2 2 2 4 4 2 2 2 4]
0.9766081871345029


In [53]:
# 5.2其他评估
ret = classification_report(y_test, y_predict, labels=(2, 4), target_names=("良性", "恶性"))
print(ret)

precision    recall  f1-score   support

          良性       0.98      0.98      0.98       111
          恶性       0.97      0.97      0.97        60

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



In [57]:
# AUC计算API
    # from sklearn.metrics import roc_auc_score
    # sklearn.metrics.roc_auc_score(y_true, y_score)
    # 计算ROC曲线面积，即AUC值
    # y_true：每个样本的真实类别，必须为0(反例),1(正例)标记
    # y_score：预测得分，可以是正类的估计概率、置信值或者分类器方法的返回值


# 不平衡二分类问题评估
y_test = np.where(y_test>3.0, 1, 0)
roc_auc_score(y_true=y_test, y_score=y_predict)

0.9743243243243243

In [ ]:
# 总结
    # AUC只能用来评价二分类
    # AUC非常适合评价样本不平衡中的分类器性能